<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/03_xyz_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-18 13:18:33--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11745 (11K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.47K  --.-KB/s    in 0s      

2022-06-18 13:18:33 (84.9 MB/s) - ‘utils.py’ saved [11745/11745]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import ModelCreator, get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 1.3 MB/s 


In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, _ = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3510 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
xyz_creator = ModelCreator(HUB_URL, 'XYZ-model')
xyz_model = xyz_creator.make_model(img_size=IMG_SIZE)

Model: "XYZ-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23500352  
                                                                 
 dense (Dense)               (None, 16)                32784     
                                                                 
Total params: 23,533,136
Trainable params: 32,784
Non-trainable params: 23,500,352
_________________________________________________________________


In [10]:
callbacks = xyz_creator.get_callbacks()

In [11]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=20,
                        class_weight=class_weight)

Epoch 1/20
519/519 [==============================] - 2819s 5s/step - loss: 115.9474 - auc: 0.4855 - accuracy: 0.5753
Epoch 2/20
519/519 [==============================] - 182s 348ms/step - loss: 79.3776 - auc: 0.6174 - accuracy: 0.7002
Epoch 3/20
519/519 [==============================] - 180s 344ms/step - loss: 71.2091 - auc: 0.6462 - accuracy: 0.7256
Epoch 4/20
519/519 [==============================] - 176s 337ms/step - loss: 53.2773 - auc: 0.6991 - accuracy: 0.7657
Epoch 5/20
519/519 [==============================] - 179s 343ms/step - loss: 45.1622 - auc: 0.7359 - accuracy: 0.7920
Epoch 6/20
519/519 [==============================] - 183s 351ms/step - loss: 37.4085 - auc: 0.7586 - accuracy: 0.8054
Epoch 7/20
519/519 [==============================] - 178s 341ms/step - loss: 35.5113 - auc: 0.7786 - accuracy: 0.8194
Epoch 8/20
519/519 [==============================] - 179s 343ms/step - loss: 29.7567 - auc: 0.7969 - accuracy: 0.8326
Epoch 9/20
519/519 [=============================

In [12]:
!cp ./logs/checkpoints/XYZ-model-10.h5 ./drive/MyDrive/checkpoints/xyz-model-10.h5
!cp ./logs/checkpoints/XYZ-model-15.h5 ./drive/MyDrive/checkpoints/xyz-model-15.h5
!cp ./logs/checkpoints/XYZ-model-20.h5 ./drive/MyDrive/checkpoints/xyz-model-20.h5

In [13]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=30,
                        initial_epoch=20,
                        class_weight=class_weight)

Epoch 21/30
519/519 [==============================] - 179s 339ms/step - loss: 3.6200 - auc: 0.9587 - accuracy: 0.9388
Epoch 22/30
519/519 [==============================] - 180s 344ms/step - loss: 3.2040 - auc: 0.9687 - accuracy: 0.9495
Epoch 23/30
519/519 [==============================] - 178s 340ms/step - loss: 3.0689 - auc: 0.9666 - accuracy: 0.9493
Epoch 24/30
519/519 [==============================] - 180s 344ms/step - loss: 2.4269 - auc: 0.9754 - accuracy: 0.9572
Epoch 25/30
519/519 [==============================] - 183s 349ms/step - loss: 2.1273 - auc: 0.9786 - accuracy: 0.9602
Epoch 26/30
519/519 [==============================] - 196s 374ms/step - loss: 1.9823 - auc: 0.9801 - accuracy: 0.9608
Epoch 27/30
519/519 [==============================] - 178s 340ms/step - loss: 1.7686 - auc: 0.9837 - accuracy: 0.9660
Epoch 28/30
519/519 [==============================] - 177s 338ms/step - loss: 1.6524 - auc: 0.9842 - accuracy: 0.9668
Epoch 29/30
519/519 [===========================

In [14]:
!cp ./logs/checkpoints/XYZ-model-25.h5 ./drive/MyDrive/checkpoints/xyz-model-25.h5
!cp ./logs/checkpoints/XYZ-model-30.h5 ./drive/MyDrive/checkpoints/xyz-model-30.h5

In [15]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=35,
                        initial_epoch=30,
                        class_weight=class_weight)

Epoch 31/35
519/519 [==============================] - 179s 343ms/step - loss: 1.1576 - auc: 0.9896 - accuracy: 0.9742
Epoch 32/35
519/519 [==============================] - 178s 340ms/step - loss: 1.0805 - auc: 0.9905 - accuracy: 0.9754
Epoch 33/35
519/519 [==============================] - 177s 337ms/step - loss: 1.0385 - auc: 0.9907 - accuracy: 0.9768
Epoch 34/35
519/519 [==============================] - 179s 343ms/step - loss: 0.9734 - auc: 0.9920 - accuracy: 0.9790
Epoch 35/35
519/519 [==============================] - 177s 338ms/step - loss: 0.8809 - auc: 0.9931 - accuracy: 0.9804


In [16]:
!cp ./logs/checkpoints/XYZ-model-35.h5 ./drive/MyDrive/checkpoints/xyz-model-35.h5